# Implementing A GPT-model From Scratch

- Coding a GPT-like LLM that can be trained to generate uman-like text.
- Normalizing layer activations to stabilize NN training.
- Adding `shortcut connections` to train deep NNs effectively.
- Implementing transformer blocks to create GPT models of various sizes.
- Computing the number of parameters and storage requiremnts of GPT models.

In [ ]:
%load_ext watermark
%watermark -v -p numpy,pandas,polars,torch,lightning --conda

In [ ]:
# Built-in library
from pathlib import Path
import re
import json
from typing import Any, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import numpy.typing as npt
from pprint import pprint
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "info": "#76FF7B",
        "warning": "#FBDDFE",
        "error": "#FF0000",
    }
)
console = Console(theme=custom_theme)

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")


# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch import nn, Tensor
import torch.nn.functional as F

In [ ]:
GPT_CONFIG_124M: dict[str, Any] = {
    "vocab_size": 50_257,
    "context_length": 1_024,
    "emb_dim": 768,
    "n_heads": 12,  # Number of attention heads
    "n_layers": 12,
    "drop_rate": 0.1,  # Dropout rate
    "qkv_bias": False,
}

### A Placeholder GPT Backbone

In [ ]:
class DummyGPTModel(nn.Module):
    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

        self.tok_emb = nn.Embedding(cfg.get("vocab_size"), cfg.get("emb_dim"))
        self.pos_emb = nn.Embedding(cfg.get("context_length"), cfg.get("emb_dim"))
        self.drop_emb = nn.Dropout(cfg.get("drop_rate"))
        self.transformer_blocks = nn.Sequential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg.get("n_layers"))]
        )

        self.final_norm = DummyLayerNorm(cfg.get("emb_dim"))
        self.out_head = nn.Linear(cfg.get("emb_dim"), cfg.get("vocab_size"), bias=False)

    def forward(self, x: Tensor) -> Tensor:
        batch_size, seq_len = x.shape
        tok_embeds: Tensor = self.tok_emb(x)
        pos_embeds: Tensor = self.pos_emb(torch.arange(seq_len, device=x.device))
        x = (
            tok_embeds + pos_embeds
        )  # Add token and positional embeddings to get input embeddings
        x = self.drop_emb(x)
        x = self.transformer_blocks(x)
        x = self.final_norm(x)
        logits: Tensor = self.out_head(x)
        return logits


class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

    def forward(self, x: Tensor) -> Tensor:
        return x


class DummyLayerNorm(nn.Module):
    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

    def forward(self, x: Tensor) -> Tensor:
        return x

### Overview

<img src="../08-Makemore/images/ch04_mental_.gpt_pic.png" alt="gpt" width="600">

[image source](https://livebook.manning.com/book/build-a-large-language-model-from-scratch/chapter-4/v-7/36)

In [ ]:
# Tokenize a batch consisting of sample texts.
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
text1: str = "Every effort moves you"
text2: str = "Everyday holds a"
batch: list[int] = []

batch.append(torch.tensor(tokenizer.encode(text1)))
batch.append(torch.tensor(tokenizer.encode(text2)))
batch_tensor: Tensor = torch.stack(batch, dim=0)

print(f"Shape of batch_tensor: {tuple(batch_tensor.shape)}\n")
print(f"Batch_tensor: \n{batch_tensor}\n")

In [ ]:
# Instantiate the DummyGPTModel
seed: int = 5
torch.manual_seed(seed)

model = DummyGPTModel(GPT_CONFIG_124M)
logits: Tensor = model(batch_tensor)

print(f"Shape of logits: {tuple(logits.shape)}\n")
print(f"logits: \n{logits}\n")

### Comment

- The output (2, 4, 50_257) has `2` rows corresponding to the number of input data.
- It has `4` rows which is the context length of the input and `50_257` is the number of unique words (vocab_size) in the input.

### Normalizing Activations With Layer Normalization

- Layer  normalization is used to improve the stability and efficiency of neural networks.
- It aims to to adjust the activations at the output of a NN layer to have a mean of 0 and a standard deviation of 1.
- These adjustments speed up the convergence to effective weights and ensures consistent, reliable training.

<br>

<img src="../08-Makemore/images/ch04_norm_layer.png" width="600">

[image source](https://livebook.manning.com/book/build-a-large-language-model-from-scratch/chapter-4/v-7/52)

<br><hr>

#### Layer Norm vs Batch Norm

- Batch norm normalizes across the batch dimension, whereas layer norm normalizes across the feature/channel dimension.
- Layer norm is preferred because it normalizes each input independently of the batch size and it's more efficient and stable.

In [ ]:
torch.manual_seed(seed)

batch_tensor: Tensor = torch.randn(2, 5)
layer = nn.Sequential(nn.Linear(5, 6), nn.ReLU())
out: Tensor = layer(batch_tensor)
print(f"out: \n{out}\n")

mean: Tensor = out.mean(dim=-1, keepdim=True)
variance: Tensor = out.var(dim=-1, unbiased=False, keepdim=True)
print("===== Before Layer Normalization ===== \n")
print(f"mean: \n{mean}\n\nvariance: \n{variance}\n")

In [ ]:
# Normalize the output
out_norm: Tensor = (out - mean) / torch.sqrt(variance)
mean_1: Tensor = out_norm.mean(dim=-1, keepdim=True)
variance_1: Tensor = out_norm.var(dim=-1, unbiased=False, keepdim=True)
print(f"out_norm: \n{out_norm}\n")
print("===== After Layer Normalization ===== \n")
print(f"mean: \n{mean_1}\n\nvariance: \n{variance_1}\n")

In [ ]:
# Improve readability
torch.set_printoptions(sci_mode=False)
print("===== After Layer Normalization ===== \n")
print(f"mean: \n{mean_1}\n\nvariance: \n{variance_1}\n")

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, embed_dim: int) -> None:
        super().__init__()

        self.eps: float = 1e-5  # Prevents zero division

        # Trainable params that's automatically adjusted by the LLM during
        # training to improve model performance
        self.scale = nn.Parameter(torch.ones(embed_dim))
        self.shift = nn.Parameter(torch.zeros(embed_dim))

    def forward(self, x: Tensor) -> Tensor:
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, unbiased=False, keepdim=True)
        norm_x: Tensor = (x - mean) / (std + self.eps)

        return (self.scale * norm_x) + self.shift

In [ ]:
layer_norm = LayerNorm(embed_dim=5)
out_layer_norm: Tensor = layer_norm(batch_tensor)

mean: Tensor = out_layer_norm.mean(dim=-1, keepdim=True)
variance: Tensor = out_layer_norm.var(dim=-1, unbiased=False, keepdim=True)
print(f"out_layer_norm: \n{out_layer_norm}\n")
print("===== After Layer Normalization ===== \n")
print(f"mean: \n{mean}\n\nvariance: \n{variance}\n")

### Implementing GELU (Gaussian Error Linear Unit)

- **Smoothness**: Unlike ReLU's sharp cutoff at zero, `GELU` has a smoother gradient transition. This smoothness helps with training, as optimization algorithms can navigate the function's landscape more effectively.

- **Non-zero gradients**: ReLU outputs zero for negative inputs, killing the gradient and hindering learning in those regions. `GELU` maintains non-zero gradients even for negative inputs, allowing the network to learn from a wider range of data. i.e. neurons with -ve gradients can still contribute to the learning process.

#### Note

- **Computational cost**: The original `GELU` formulation can be computationally expensive. Luckily, there are efficient approximations that retain the benefits without the high cost.

<!-- [![image.png](https://i.postimg.cc/9XSGKNZ9/image.png)](https://postimg.cc/14GnNBWR) -->
[![image.png](https://i.postimg.cc/pTrF1Ddk/image.png)](https://postimg.cc/wyKM1RLs)

$$GELU(x) = 0.5x * ( 1 + tanh( \sqrt{\frac{2}{\pi}} * (x + 0.044715x^{3}) ) ) $$

In [ ]:
class GELU(nn.Module):
    def __init__(self) -> None:
        super().__init__()

    def forward(self, x: Tensor) -> Tensor:
        gelu: Tensor = (
            0.5
            * x
            * (
                1
                + torch.tanh(
                    torch.sqrt(torch.tensor(2.0 / torch.pi))
                    * (x + 0.044715 * torch.pow(x, 3))
                )
            )
        )
        return gelu


class FeedForward(nn.Module):
    """Applies a feed-forward neural network to the input tensor `x`. The feed-forward network
    consists of two linear layers with a GELU activation in between. The first linear layer
    expands the input dimension by a factor of 4, and the second linear layer projects the result
    back to the original input dimension.

    Args:
        x (torch.Tensor): The input tensor to be passed through the feed-forward network.

    Returns:
        torch.Tensor: The output tensor after passing through the feed-forward network.
    """

    def __init__(self, cfg: dict[str, Any]) -> None:
        super().__init__()

        self.layers = nn.Sequential(
            nn.Linear(cfg.get("emb_dim"), 4 * cfg.get("emb_dim")),
            GELU(),
            nn.Linear(4 * cfg.get("emb_dim"), cfg.get("emb_dim")),
        )

    def forward(self, x: Tensor) -> Tensor:
        return self.layers(x)

In [ ]:
gelu, relu = GELU(), nn.ReLU()

x: Tensor = torch.linspace(-3, 3, 100)
y_gelu, y_relu = gelu(x), relu(x)
plt.figure(figsize=(8, 3))

for i, (y, label) in enumerate(zip([y_gelu, y_relu], ["GELU", "ReLU"]), 1):
    plt.subplot(1, 2, i)
    plt.plot(x, y)
    plt.title(f"{label} activation function")
    plt.xlabel("x")
    plt.ylabel(f"{label}(x)")
    plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
torch.manual_seed(seed)

ff_network = FeedForward(cfg=GPT_CONFIG_124M)
x: Tensor = torch.randn(2, 3, 768)
out: Tensor = ff_network(x)

print(f"out: {out.shape}")
print(f"out: \n{out}")

#### Feed Forward Network With GELU

<img src="../08-Makemore/images/ch04__ffn.png" width="600">

[image source](https://livebook.manning.com/book/build-a-large-language-model-from-scratch/chapter-4/v-7/112)

### Shortcut Connections

- AKA Skip/Residual connections.
- Shortcut Connections involve adding the output of one layer to the input of the next layer.
- It mitigates the vanishing gradient problem.

<img src="../08-Makemore/images/ch04__shortcut.png" width="600">

[image source](https://livebook.manning.com/book/build-a-large-language-model-from-scratch/chapter-4/v-7/119)

In [ ]:
class ExampleDeepNeuralNetwork_v1(nn.Module):
    def __init__(self, layer_sizes: list[int], use_shortcut: bool = True):
        super().__init__()

        self.use_shortcut = use_shortcut
        assert len(layer_sizes) == 6, "layer_sizes must be of length 6"
        self.layers = nn.ModuleList(
            [
                # 5 layers
                nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
                nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU()),
            ]
        )

    def forward(self, x: Tensor) -> Tensor:
        for layer in self.layers:
            # Compute the output of the current layer
            layer_output: Tensor = layer(x)
            # Check if shortcut connections can be applied
            if self.use_shortcut and x.shape == layer_output.shape:
                # Apply the shortcut connection
                x = x + layer_output
            else:
                # Otherwise, just use the layer output
                x = layer_output
        return x


def print_gradients(model: nn.Module, x: Tensor) -> None:
    """This is used to print the gradients of the model."""
    # Forward pass
    output: Tensor = model(x)
    target: Tensor = torch.tensor([[0.0]])

    # Calculate the loss
    criterion = nn.MSELoss()
    loss: Tensor = criterion(output, target)

    # Backward pass
    loss.backward()

    for name, param in model.named_parameters():
        if "weight" in name:
            print(f"{name} has gradient mean of {param.grad.abs().mean().item():.6f}")

In [ ]:
[f"Linear({idx}, {idx + 1})" for idx in range(6 - 1)]

In [ ]:
torch.manual_seed(seed)

layer_sizes: list[int] = [3, 3, 3, 3, 3, 1]
sample_input: Tensor = torch.tensor([[1.0, 0.0, -1.0]])

model_wfout_shortcut = ExampleDeepNeuralNetwork_v1(
    layer_sizes=layer_sizes, use_shortcut=False
)
print(f"input: {sample_input.shape}")

model_wfout_shortcut

In [ ]:
print_gradients(model=model_wfout_shortcut, x=sample_input)

In [ ]:
class ExampleDeepNeuralNetwork_v2(nn.Module):
    def __init__(self, layer_sizes: list[int], use_shortcut: bool = True):
        super().__init__()

        self.use_shortcut = use_shortcut
        assert len(layer_sizes) == 6, "layer_sizes must be of length 6"
        self.layers = nn.ModuleList(
            [
                # 5 layers
                nn.Sequential(nn.Linear(layer_sizes[idx], layer_sizes[idx + 1]), GELU())
                for idx in range(len(layer_sizes) - 1)
            ]
        )

    def forward(self, x: Tensor) -> Tensor:
        for layer in self.layers:
            # Compute the output of the current layer
            layer_output: Tensor = layer(x)
            # Check if shortcut connections can be applied
            if self.use_shortcut and x.shape == layer_output.shape:
                # Apply the shortcut connection
                x = x + layer_output
            else:
                # Otherwise, just use the layer output
                x = layer_output
        return x

In [ ]:
torch.manual_seed(seed)

layer_sizes: list[int] = [3, 3, 3, 3, 3, 1]
sample_input: Tensor = torch.tensor([[1.0, 0.0, -1.0]])

model_wfout_shortcut = ExampleDeepNeuralNetwork_v2(
    layer_sizes=layer_sizes, use_shortcut=False
)
print(f"input: {sample_input.shape}")
print(model_wfout_shortcut)
print()

# The weights shrink to a vanishingly small value
print_gradients(model=model_wfout_shortcut, x=sample_input)

In [ ]:
torch.manual_seed(seed)


model_wf_shortcut = ExampleDeepNeuralNetwork_v2(
    layer_sizes=layer_sizes,
    use_shortcut=True,  # NEW!
)
print(f"input: {sample_input.shape}")
print(model_wf_shortcut)
print()

# The weights are slightly larger than the ones without the shortcut.
print_gradients(model=model_wf_shortcut, x=sample_input)

<br><br>

### Connecting Attention And Linear Layers In A Transformer Block

- Implement the transformer block.
- The transformer block in the 124 million parameter GPT-2 architecture was repeated 12 times and it combines several concepts like:
  - multi-head attention
  - layer norm, dropout
  - feedforward layers
  - and GELU activations.

<br>

<img src="../08-Makemore/images/ch04__trnsf_block.png" width="600">

[image source](https://livebook.manning.com/book/build-a-large-language-model-from-scratch/chapter-4/v-7/145)

<br>

- When a transformer block processes an input sequence, each element in the sequence is represented by a fixed-size vector.
- The operations w/in the transformer block are designed to transform the input vectors in a way that preserves the dimensions.
- The multi-head attention block figures out how different parts of the sequence relate to each other, while the feedforward network modifies the data individually at each position.
- This combination leads to a more nuanced understanding and processing of the input and improves the model's capacity to handle complex data patterns.